In [ ]:
import os
import torch

from torchvision.io import read_image
from torchvision.ops.boxes import masks_to_boxes
from torchvision import tv_tensors
from torchvision.transforms.v2 import functional as F
from google.colab import drive
from PIL import Image
import matplotlib.pyplot as plt
import sys
import pandas as pd
from collections import defaultdict
import json
import random
from torch.utils.data import Dataset
import numpy as np
import torchvision.models.detection as detection
from torch.utils.data import DataLoader
import torch.utils
import torch
from PIL import Image as read_image
from torchvision.io import read_image
from torchvision.transforms import v2
from transforms import RandomPhotometricDistort, RandomZoomOut, RandomIoUCrop, RandomHorizontalFlip,RandomVerticalFlip ,ToTensor, Compose
from torchvision.transforms.functional import convert_image_dtype

In [ ]:
drive.mount('/content/drive')
path_base = '/content/drive/MyDrive/Academico/TD8 personal '
sys.path.append(path_base)

Mounted at /content/drive


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:

train_file = path_base + '/classificationDownload (1)/train.csv'
test_file = path_base + '/classificationDownload (1)/test.csv'
classification_file = path_base + '/classificationDownload (1)/classifications.csv'

train_images = pd.read_csv(train_file)['image_filename'].tolist()
test_images = pd.read_csv(test_file)['image_filename'].tolist()


classification_data = pd.read_csv(classification_file)

train_data = classification_data[classification_data['image_filename'].isin(train_images)]
test_data = classification_data[classification_data['image_filename'].isin(test_images)]




def get_transform(train=True):
    transforms = []
    if train:
        transforms.append(RandomPhotometricDistort())
        transforms.append(RandomZoomOut(fill=list((207., 202., 205.))))
        transforms.append(RandomHorizontalFlip(p=0.5))
        transforms.append(RandomVerticalFlip(p=0.5))

    transforms.append(v2.ToDtype(torch.float, scale=True))
    return Compose(transforms)


def group_detections(data):
    grouped_data = {}
    for _, row in data.iterrows():
        image_name = row['image_filename']
        if image_name not in grouped_data:
            grouped_data[image_name] = {
                'image_filename': image_name,
                'image_id': row['image_id'],
                'image_doi': row['image_doi'],
                'classifications': []
            }

        grouped_data[image_name]['classifications'].append({
            'nucleus_x': row['nucleus_x'],
            'nucleus_y': row['nucleus_y'],
            'bethesda_system': row['bethesda_system'],
            'cell_id': row['cell_id']
        })
    return list(grouped_data.values())


train_data_grouped = group_detections(train_data)
test_data_grouped = group_detections(test_data)


train_counts = defaultdict(int)
test_counts = defaultdict(int)

for item in train_data_grouped:
    for classification in item['classifications']:
        train_counts[classification['bethesda_system']] += 1

for item in test_data_grouped:
    for classification in item['classifications']:
        test_counts[classification['bethesda_system']] += 1

class PAPDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

        self.label_map = {
            "SCC": 1,
            "HSIL": 2,
            "LSIL": 3,
            "ASC-H": 4,
            "ASC-US": 5,
            "Negative for intraepithelial lesion": 6
        }

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        img = read_image(path_base + '/Imagenes/' + item['image_filename'])

        image_doi = item['image_doi']
        image_id = item['image_id']
        boxes = []
        labels = []
        for classification in item['classifications']:
            nucleus_x = classification['nucleus_x']
            nucleus_y = classification['nucleus_y']
            x_min = nucleus_x - 50
            y_min = nucleus_y - 50
            x_max = nucleus_x + 50
            y_max = nucleus_y + 50
            boxes.append([x_min, y_min, x_max, y_max])
            labels.append(self.label_map[classification['bethesda_system']])


        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])

        target = {
            "boxes": boxes,
            "labels": labels,
            "area": area,
            "image_id": image_id
        }

        num_objs = boxes.shape[0]
        target["iscrowd"] = torch.zeros((num_objs,), dtype=torch.int64)

        if self.transform:
            img, target = self.transform(img, target)
        return img, target



train_dataset = PAPDataset(train_data_grouped, transform=get_transform(train=True))
test_dataset = PAPDataset(test_data_grouped, transform=get_transform(train=False))


print("Training Counts:", dict(train_counts))
print("Test Counts:", dict(test_counts))
print(f"Training Dataset Size: {len(train_dataset)}")
print(f"Test Dataset Size: {len(test_dataset)}")


Training Counts: {'SCC': 145, 'Negative for intraepithelial lesion': 6043, 'LSIL': 1221, 'HSIL': 1588, 'ASC-H': 808, 'ASC-US': 550}
Test Counts: {'ASC-H': 117, 'Negative for intraepithelial lesion': 736, 'ASC-US': 56, 'LSIL': 139, 'SCC': 16, 'HSIL': 115}
Training Dataset Size: 360
Test Dataset Size: 40


In [ ]:
from ssd import SSDFeatureExtractorResNet_Upgraded, ssd512_resnet50_Upgraded
model = ssd512_resnet50_Upgraded(pretrained = False, load = True)
model = model.to(device)



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/content/drive/MyDrive/Academico/TD8 personal /ssd.py:1837: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. 

In [ ]:
from ssd import SSDClassificationHead, SSDRegressionHead
num_classes = 6+1  # Ejemplo con 6  clases + 1 clase background


in_channels = [8192, 3584, 3584, 3328, 256, 256]
num_anchors = [4, 6, 6, 6, 4, 4]
model.head.classification_head = SSDClassificationHead(in_channels, num_anchors, num_classes)


In [ ]:

def collate_fn(batch):
    return tuple(zip(*batch))



batch_size = 16

train_sampler = torch.utils.data.RandomSampler(train_dataset)
test_sampler = torch.utils.data.SequentialSampler(test_dataset)

train_batch_sampler = train_batch_sampler = torch.utils.data.BatchSampler(
            train_sampler, batch_size, drop_last=True)


data_loader = torch.utils.data.DataLoader(
        train_dataset, batch_sampler=train_batch_sampler,
        collate_fn=collate_fn)


data_loader_test = torch.utils.data.DataLoader(
        test_dataset, batch_size=1,
        sampler=test_sampler,
        collate_fn=collate_fn)

In [ ]:
import math
from utils import MetricLogger, warmup_lr_scheduler, SmoothedValue
def train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq):
    model.train()
    metric_logger = MetricLogger(delimiter="  ")
    metric_logger.add_meter('lr', SmoothedValue(window_size=1, fmt='{value:.6f}'))
    header = 'Epoch: [{}]'.format(epoch)

    lr_scheduler = None
    if epoch == 0:
        warmup_factor = 1. / 1000
        warmup_iters = min(1000, len(data_loader) - 1)

        lr_scheduler = warmup_lr_scheduler(optimizer, warmup_iters, warmup_factor)

    for images, targets in metric_logger.log_every(data_loader, print_freq, header):
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) if not isinstance(v, int) else v for k, v in t.items()} for t in targets]
        loss_dict = model(images, targets)

        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()
        print(loss_value)

        if not math.isfinite(loss_value):
            print("Loss is {}, stopping training".format(loss_value))
            break

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if lr_scheduler is not None:
            lr_scheduler.step()

        metric_logger.update(lr=optimizer.param_groups[0]["lr"])

    return metric_logger


'\nMetricLogger te ayuda a monitorear métricas a lo largo del entrenamiento.\nWarmup Learning Rate Scheduler te permite empezar el entrenamiento de manera más suave ajustando gradualmente la tasa de aprendizaje durante las primeras iteraciones para evitar inestabilidad.\n'

In [ ]:
!wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/engine.py
!wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/utils.py
!wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_utils.py
!wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_eval.py
!wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/transforms.py
!pip install -U pycocotools

--2024-11-19 18:00:17--  https://raw.githubusercontent.com/pytorch/vision/main/references/detection/engine.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4063 (4.0K) [text/plain]
Saving to: ‘engine.py’

engine.py           100%[===================>]   3.97K  --.-KB/s    in 0s      

2024-11-19 18:00:17 (61.2 MB/s) - ‘engine.py’ saved [4063/4063]

--2024-11-19 18:00:17--  https://raw.githubusercontent.com/pytorch/vision/main/references/detection/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8388 (8.2K) [text/plain]
S

In [ ]:
from torchvision.datasets import VOCDetection
from torchvision.transforms import Compose, ToTensor, RandomHorizontalFlip, Normalize
from engine import evaluate
import time
import datetime


params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(
        params, lr = 0.003, momentum=0.9, weight_decay = 0.0005)

lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[80,110], gamma=0.1)
model.to(device)
best_ap = 0.0

print("Start training")
start_time = time.time()
for epoch in range(50):
  train_one_epoch(model, optimizer, data_loader, device, epoch, 20)
  lr_scheduler.step()
  ap = evaluate(model, data_loader_test, device=device).coco_eval["bbox"].stats[0]
  if ap > best_ap:
    best_ap = ap
    best_model_weights = model.state_dict()

total_time = time.time() - start_time
total_time_str = str(datetime.timedelta(seconds=int(total_time)))
print('Training time {}'.format(total_time_str))








Start training
25.11927032470703
Epoch: [0]  [ 0/22]  eta: 0:03:45  lr: 0.000146  time: 10.2313  data: 6.6201  max mem: 11273
26.41074562072754
22.71268653869629
18.994705200195312
13.703094482421875
16.671476364135742
14.634176254272461
11.086359024047852
10.561686515808105
10.792121887207031
9.97930908203125
8.419669151306152
8.797271728515625
8.854043960571289
9.148704528808594
8.926671028137207
8.093688011169434
7.526572227478027
8.723756790161133
7.850994110107422
7.521486282348633
Epoch: [0]  [20/22]  eta: 0:00:12  lr: 0.003000  time: 6.1743  data: 5.8091  max mem: 12463
8.218050003051758
Epoch: [0]  [21/22]  eta: 0:00:06  lr: 0.003000  time: 6.0990  data: 5.7386  max mem: 12463
Epoch: [0] Total time: 0:02:19 (6.3192 s / it)
creating index...
index created!
Test:  [ 0/40]  eta: 0:00:16  model_time: 0.3129 (0.3129)  evaluator_time: 0.0303 (0.0303)  time: 0.4096  data: 0.0627  max mem: 12463
Test:  [39/40]  eta: 0:00:00  model_time: 0.0166 (0.0242)  evaluator_time: 0.0192 (0.0377) 

In [ ]:
print('El AP del mejor modelo siendo esre el que guardamos es:')
print(best_ap)
torch.save({
    'model_state_dict': best_model_weights,
}, path_base + "/pesos_modelos/SSD512_ResNet_upgraded_finetuned.pth" )

El AP del mejor modelo siendo esre el que guardamos es:
0.2680958707813942


In [ ]:
start_time = time.time()


for epoch in range(50, 100):
    train_one_epoch(model, optimizer, data_loader, device, epoch, 20)
    lr_scheduler.step()
    ap = evaluate(model, data_loader_test, device=device).coco_eval["bbox"].stats[0]
    if ap > best_ap:
      best_ap = ap
      best_model_weights = model.state_dict()

total_time = time.time() - start_time
total_time_str = str(datetime.timedelta(seconds=int(total_time)))
print('Training time {}'.format(total_time_str))

2.295315742492676
Epoch: [50]  [ 0/22]  eta: 0:00:52  lr: 0.003000  time: 2.3637  data: 2.0405  max mem: 12885
2.447047233581543
2.1442601680755615
2.8514864444732666
2.0185482501983643
2.2891268730163574
2.7347404956817627
2.1174933910369873
2.2802228927612305
2.542239189147949
2.097573757171631
2.104536533355713
2.873229742050171
2.5813021659851074
2.1490588188171387
2.6637954711914062
2.2578253746032715
2.1996803283691406
2.472625732421875
2.072277069091797
1.928715705871582
Epoch: [50]  [20/22]  eta: 0:00:04  lr: 0.003000  time: 2.1893  data: 1.8573  max mem: 12885
2.287635564804077
Epoch: [50]  [21/22]  eta: 0:00:02  lr: 0.003000  time: 2.1504  data: 1.8218  max mem: 12885
Epoch: [50] Total time: 0:00:48 (2.2072 s / it)
creating index...
index created!
Test:  [ 0/40]  eta: 0:00:04  model_time: 0.0170 (0.0170)  evaluator_time: 0.0200 (0.0200)  time: 0.1021  data: 0.0615  max mem: 12885
Test:  [39/40]  eta: 0:00:00  model_time: 0.0165 (0.0165)  evaluator_time: 0.0192 (0.0295)  time:

In [ ]:
print('El AP del mejor modelo siendo esre el que guardamos es:')
print(best_ap)
torch.save({
    'model_state_dict': best_model_weights,
}, path_base + "/pesos_modelos/SSD512_ResNet_upgraded_finetuned.pth"   )

El AP del mejor modelo siendo esre el que guardamos es:
0.29881452517346024


In [ ]:
start_time = time.time()


for epoch in range(100, 120):
    train_one_epoch(model, optimizer, data_loader, device, epoch, 20)
    lr_scheduler.step()
    ap = evaluate(model, data_loader_test, device=device).coco_eval["bbox"].stats[0]
    if ap > best_ap:
      best_ap = ap
      best_model_weights = model.state_dict()

total_time = time.time() - start_time
total_time_str = str(datetime.timedelta(seconds=int(total_time)))
print('Training time {}'.format(total_time_str))

1.9080226421356201
Epoch: [100]  [ 0/22]  eta: 0:00:52  lr: 0.000300  time: 2.3722  data: 2.0531  max mem: 12892
1.554222583770752
1.818747878074646
1.7284327745437622
1.759906530380249
1.6784331798553467
1.7957497835159302
1.8905892372131348
1.3840364217758179
1.746946930885315
1.6150803565979004
2.00706148147583
1.7305346727371216
1.5604149103164673
1.998816967010498
1.8235204219818115
2.0636861324310303
1.9379171133041382
1.7314600944519043
1.7827610969543457
2.5643861293792725
Epoch: [100]  [20/22]  eta: 0:00:04  lr: 0.000300  time: 2.1744  data: 1.8105  max mem: 12892
1.7729945182800293
Epoch: [100]  [21/22]  eta: 0:00:02  lr: 0.000300  time: 2.1719  data: 1.8079  max mem: 12892
Epoch: [100] Total time: 0:00:47 (2.1814 s / it)
creating index...
index created!
Test:  [ 0/40]  eta: 0:00:03  model_time: 0.0173 (0.0173)  evaluator_time: 0.0128 (0.0128)  time: 0.0946  data: 0.0609  max mem: 12892
Test:  [39/40]  eta: 0:00:00  model_time: 0.0162 (0.0163)  evaluator_time: 0.0174 (0.0262)

In [ ]:
start_time = time.time()

for epoch in range(120, 150):
    train_one_epoch(model, optimizer, data_loader, device, epoch, 20)
    lr_scheduler.step()
    ap = evaluate(model, data_loader_test, device=device).coco_eval["bbox"].stats[0]
    if ap > best_ap:
      best_ap = ap
      best_model_weights = model.state_dict()

total_time = time.time() - start_time
total_time_str = str(datetime.timedelta(seconds=int(total_time)))
print('Training time {}'.format(total_time_str))

1.8815008401870728
Epoch: [120]  [ 0/22]  eta: 0:00:52  lr: 0.000030  time: 2.3955  data: 2.1442  max mem: 12999
1.765442132949829
1.4625215530395508
2.1458537578582764
2.3577799797058105
1.645620584487915
1.9083654880523682
1.7686976194381714
1.7786095142364502
1.8517048358917236
2.361187696456909
1.5881495475769043
1.993222713470459
1.6873246431350708
1.5787760019302368
1.9499197006225586
1.8770813941955566
1.7296322584152222
2.80816650390625
2.0396387577056885
1.620174765586853
Epoch: [120]  [20/22]  eta: 0:00:04  lr: 0.000030  time: 2.2184  data: 1.8572  max mem: 12999
1.5713119506835938
Epoch: [120]  [21/22]  eta: 0:00:02  lr: 0.000030  time: 2.2092  data: 1.8490  max mem: 12999
Epoch: [120] Total time: 0:00:48 (2.2231 s / it)
creating index...
index created!
Test:  [ 0/40]  eta: 0:00:03  model_time: 0.0171 (0.0171)  evaluator_time: 0.0128 (0.0128)  time: 0.0968  data: 0.0632  max mem: 12999
Test:  [39/40]  eta: 0:00:00  model_time: 0.0164 (0.0165)  evaluator_time: 0.0230 (0.0299)